<a href="https://colab.research.google.com/github/cerapcapiq/radio_app1/blob/main/radioapptest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check TensorFlow version
import tensorflow as tf
print(tf.__version__)

# Import libraries
import numpy as np
import matplotlib.pyplot as plt

# You can download datasets using code, e.g., MedNIST
!wget -q --show-progress https://example.com/MedNIST.zip



2.18.0


In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import zipfile
import urllib.request
import tarfile

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# Download MedNIST dataset (small subset with X-ray and MRI)
import tarfile

data_url = "https://github.com/Project-MONAI/MONAI-extra-test-data/releases/download/0.8.1/MedNIST.tar.gz"
data_dir = pathlib.Path("MedNIST")

if not data_dir.exists():
    # Download the tar.gz file
    tar_path, _ = urllib.request.urlretrieve(data_url, "MedNIST.tar.gz")
    # Extract it
    with tarfile.open("MedNIST.tar.gz", "r:gz") as tar_ref:
        tar_ref.extractall()


# Check directory structure
print(f"Data directory exists: {data_dir.exists()}")
print(f"Contents: {os.listdir(data_dir)}")

# Focus on 'ChestCT' (MRI) and 'ChestXRay' (X-ray) folders
classes = ['ChestCT', 'CXR']
image_paths = []
labels = []

for label, class_name in enumerate(classes):
    class_dir = data_dir / class_name
    for img_file in os.listdir(class_dir):
        image_paths.append(str(class_dir / img_file))
        labels.append(label)

print(f"Total images collected: {len(image_paths)}")

# Shuffle data
from sklearn.utils import shuffle
image_paths, labels = shuffle(image_paths, labels, random_state=42)

# Load and preprocess images
IMG_SIZE = 64

def preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=1)  # grayscale
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = img / 255.0
    return img

# Create TensorFlow dataset
AUTOTUNE = tf.data.AUTOTUNE
path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
image_ds = path_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(labels)
dataset = tf.data.Dataset.zip((image_ds, label_ds))

# Split dataset: 80% train, 20% test
dataset_size = len(image_paths)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

train_ds = dataset.take(train_size).batch(32).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = dataset.skip(train_size).batch(32).cache().prefetch(buffer_size=AUTOTUNE)

# Build simple CNN model
model = models.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(classes), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train model
history = model.fit(train_ds, epochs=5, validation_data=test_ds)

# Evaluate model on test set
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.3f}")

# Save the model
model.save("mednist_cnn_model")

# Test prediction example
for images, labels in test_ds.take(1):
    preds = model.predict(images)
    for i in range(3):
        plt.imshow(tf.squeeze(images[i]), cmap='gray')
        plt.title(f"True: {classes[labels[i]]}\nPredicted: {classes[np.argmax(preds[i])]}")
        plt.axis('off')
        plt.show()



TensorFlow version: 2.18.0
Data directory exists: True
Contents: ['CXR', 'AbdomenCT', 'ChestCT', 'Hand', 'BreastMRI', 'HeadCT', 'README.md']
Total images collected: 20000


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 821,826 (3.14 MB)

 Trainable params: 821,826 (3.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 81s 158ms/step - accuracy: 0.9579 - loss: 0.0998 - val_accuracy: 1.0000 - val_loss: 6.7673e-06
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 73s 146ms/step - accuracy: 1.0000 - loss: 6.1693e-05 - val_accuracy: 1.0000 - val_loss: 1.5570e-06
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 72s 143ms/step - accuracy: 1.0000 - loss: 2.5348e-06 - val_accuracy: 1.0000 - val_loss: 8.5510e-07
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 84s 148ms/step - accuracy: 1.0000 - loss: 2.1909e-06 - val_accuracy: 1.0000 - val_loss: 6.3737e-07
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 80s 143ms/step - accuracy: 1.0000 - loss: 1.4879e-06 - val_accuracy: 1.0000 - val_loss: 4.7653e-07
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 1.0000 - loss: 5.0004e-07
Test accuracy: 1.000


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=mednist_cnn_model.